In [1]:
print("hi_world")

hi_world


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sna
from sklearn.metrics import mean_squared_error , r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings




In [3]:
df = pd.read_csv('data/stud.csv')

In [4]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [5]:
x = df.drop('math_score',axis=1)

In [6]:
x.shape

(1000, 7)

In [7]:
y = df['math_score']

In [8]:
y.shape

(1000,)

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [10]:
catagarical_fetures = x_train.select_dtypes(include=['object']).columns
numerical_features = x_train.select_dtypes(exclude=['object']).columns

In [11]:
for col in catagarical_fetures:
    print(f"{col} : {df[col].unique()}")

gender : ['female' 'male']
race_ethnicity : ['group B' 'group C' 'group A' 'group D' 'group E']
parental_level_of_education : ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
lunch : ['standard' 'free/reduced']
test_preparation_course : ['none' 'completed']


In [12]:
for num in numerical_features:
    print(f"{num} : {df[num].unique()}")

reading_score : [ 72  90  95  57  78  83  43  64  60  54  52  81  53  75  89  32  42  58
  69  73  71  74  70  65  87  56  61  84  55  44  41  85  59  17  39  80
  37  63  51  49  26  68  45  47  86  34  79  66  67  91 100  76  77  82
  92  93  62  88  50  28  48  46  23  38  94  97  99  31  96  24  29  40]
writing_score : [ 74  88  93  44  75  78  92  39  67  50  52  43  73  70  58  86  28  46
  61  63  53  80  72  55  65  38  82  79  83  59  57  54  68  66  62  76
  48  42  87  49  10  34  71  37  56  41  22  81  45  36  89  47  90 100
  64  98  51  40  84  69  33  60  85  91  77  27  94  95  19  35  32  96
  97  99  15  30  23]


In [13]:
from sklearn.preprocessing import OneHotEncoder ,StandardScaler
from sklearn.compose import ColumnTransformer

In [14]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [15]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, catagarical_fetures)
    ]
)

In [16]:
preprocessor.fit(x_train)

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


In [17]:
X_train_processed = preprocessor.transform(x_train)
X_test_processed = preprocessor.transform(x_test)

In [18]:
X_train_processed

array([[ 0.03079054,  0.43405338,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 0.9302895 ,  0.96470125,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.34544287,  1.1636942 ,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.52274728, -1.02522827,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.49143847, -1.02522827,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.48382733,  1.36268716,  1.        , ...,  1.        ,
         0.        ,  1.        ]], shape=(800, 19))

In [19]:
X_test_processed

array([[ 1.13786619,  1.03103224,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [-0.24597837,  0.30139141,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.23836723,  0.23506043,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       ...,
       [ 0.79190505,  0.50038436,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.51513614,  0.36772239,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 0.30755945,  0.23506043,  1.        , ...,  1.        ,
         1.        ,  0.        ]], shape=(200, 19))

In [20]:
X_train_processed.shape, X_test_processed.shape

((800, 19), (200, 19))

In [21]:
X_train_processed

array([[ 0.03079054,  0.43405338,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 0.9302895 ,  0.96470125,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.34544287,  1.1636942 ,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.52274728, -1.02522827,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.49143847, -1.02522827,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.48382733,  1.36268716,  1.        , ...,  1.        ,
         0.        ,  1.        ]], shape=(800, 19))

In [22]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [23]:

models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train_processed, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train_processed)
    y_test_pred = model.predict(X_test_processed)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.3231
- Mean Absolute Error: 4.2667
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3940
- Mean Absolute Error: 4.2148
- R2 Score: 0.8804


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5925
- Mean Absolute Error: 5.2053
- R2 Score: 0.8072
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.5173
- Mean Absolute Error: 5.1557
- R2 Score: 0.8254


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.3233
- Mean Absolute Error: 4.2650
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3904
- Mean Absolute Error: 4.2111
- R2 Score: 0.8806


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.6991
- Mean Absolute Error: 4.5083
- R2 Score: 0.8559
-----------------------

In [24]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
2,Ridge,0.880592
0,Linear Regression,0.880433
5,Random Forest Regressor,0.852426
7,CatBoosting Regressor,0.851831
8,AdaBoost Regressor,0.845719
1,Lasso,0.825447
6,XGBRegressor,0.821221
3,K-Neighbors Regressor,0.783958
4,Decision Tree,0.755012


In [25]:
for i in range (len(models)):
    print(i)
  

0
1
2
3
4
5
6
7
8


In [26]:
for i in range(len(list(models))):
    model5 = list(models.values())[i]
    print(model5)

LinearRegression()
Lasso()
Ridge()
KNeighborsRegressor()
DecisionTreeRegressor()
RandomForestRegressor()
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)
AdaBoostRegressor()


In [27]:
list(models.values())[0]

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [28]:
[list(models.values())[i] for i in range(len(models))]

[LinearRegression(),
 Lasso(),
 Ridge(),
 KNeighborsRegressor(),
 DecisionTreeRegressor(),
 RandomForestRegressor(),
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...),
 AdaBoostRegressor()]

In [29]:
x=[i for i in range(len(models))]

In [30]:
model_instances = [list(models.values())[i] for i in range(len(models))]

In [31]:
model_instances

[LinearRegression(),
 Lasso(),
 Ridge(),
 KNeighborsRegressor(),
 DecisionTreeRegressor(),
 RandomForestRegressor(),
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...),
 AdaBoostRegressor()]